In [1]:
%pwd

'd:\\OneDrive - Hanoi University of Science and Technology\\HUST\\CLASS\\20211\\DataScience\\Project\\Test_alonhadat\\pre-processing'

In [2]:
%cd ..

d:\OneDrive - Hanoi University of Science and Technology\HUST\CLASS\20211\DataScience\Project\Test_alonhadat


In [3]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np 



In [4]:
spark = SparkSession.builder.appName('Clean Data').getOrCreate()

In [5]:
schema = StructType([
    StructField('address', StringType(), True),
    StructField('facade', FloatType(), True),
    StructField('direction', StringType(), True),
    StructField('area', FloatType(), True),
    StructField('bedroom', IntegerType(), True),
    StructField('bathroom', IntegerType(), True),
    StructField('floors', IntegerType(), True),
    StructField('house_type', StringType(), True),
    StructField('price', FloatType(), True),
    StructField('street_size', FloatType(), True),
    StructField('lat', FloatType(), True),
    StructField('lng', FloatType(), True),
])



In [6]:
spark

In [16]:
bds_df = spark.read.csv("resources/data/final/alo_nha_dat_lat_long.csv", header=True)

Convert Type

In [17]:
cvt = {"StringType" : 'str',
 "FloatType" : 'float',
 "IntegerType" : 'int'}

sql_cvt = [f"cast({field.name} as {cvt[str(field.dataType)]})  {field.name}" if str(field.dataType) != 'StringType' else f"{field.name}" for field in schema.fields ]

bds_df_cast = bds_df.selectExpr(*sql_cvt)
bds_df_cast = bds_df_cast.sample(0.4)

In [18]:
print("Number of rows: ", bds_df_cast.count())
bds_df_cast.printSchema()
bds_df_cast.show()



Number of rows:  23814
root
 |-- address: string (nullable = true)
 |-- facade: float (nullable = true)
 |-- direction: string (nullable = true)
 |-- area: float (nullable = true)
 |-- bedroom: integer (nullable = true)
 |-- bathroom: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- house_type: string (nullable = true)
 |-- price: float (nullable = true)
 |-- street_size: float (nullable = true)
 |-- lat: float (nullable = true)
 |-- lng: float (nullable = true)

+--------------------+------+---------+-----+-------+--------+------+--------------------+-----+-----------+--------+---------+
|             address|facade|direction| area|bedroom|bathroom|floors|          house_type|price|street_size|     lat|      lng|
+--------------------+------+---------+-----+-------+--------+------+--------------------+-----+-----------+--------+---------+
| Đường Lê Trọng T...|   4.0| Đông Nam| 31.0|      3|    null|     3|        Nhà mặt tiền|  1.5|        3.0| 20.9629|105.76203|

Remove some null value

In [19]:
# remove null address, price or lat/lng

bds_df_cast = bds_df_cast.filter("address is not NULL")\
    .filter("price is not NULL")\
    .filter("lat != 0.0")\
    .filter("area is not NULL")
    
print("Number of rows after filter: ", bds_df_cast.count())

Number of rows after filter:  23758


In [20]:
bds_df_cast = bds_df_cast.sample(0.3)

In [21]:
bds_df_cast.describe().show()

+-------+--------------------+------------------+---------+-----------------+-----------------+--------+------------------+--------------------+-----------------+------------------+--------------------+------------------+
|summary|             address|            facade|direction|             area|          bedroom|bathroom|            floors|          house_type|            price|       street_size|                 lat|               lng|
+-------+--------------------+------------------+---------+-----------------+-----------------+--------+------------------+--------------------+-----------------+------------------+--------------------+------------------+
|  count|                7088|              5141|      409|             7088|             6600|       0|              6648|                7088|             7088|              7018|                7088|              7088|
|   mean|                null| 6.468799837424908|     null|83.84149647292179|5.586515151515152|    null| 4.94344

In [23]:
bds_df_cast.toPandas().to_csv("resources/data/final/clean/alo_nha_dat_lat_long_clean.csv", header=True)